<img src = "https://github.com/singlestore-labs/spaces-notebooks/blob/e551e274bb67bb1e5081131ee1150cdba713fc43/common/images/singlestore-jupyter.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Apache Spark + OpenAI for Personalized Banking Services, Part 3</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!conda install -y --quiet -c conda-forge openjdk pyspark

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
!pip uninstall langchain-openai -y --quiet
!pip install openai==0.28 --quiet
!pip install nltk --quiet

In [7]:
import getpass
import openai

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [8]:
import os

os.makedirs("jars", exist_ok = True)
os.makedirs("data", exist_ok = True)

In [9]:
import requests

def download_jar(url, destination):
    response = requests.get(url)
    with open(destination, "wb") as f:
        f.write(response.content)

jar_urls = [
    ("https://repo1.maven.org/maven2/com/singlestore/singlestore-jdbc-client/1.2.1/singlestore-jdbc-client-1.2.1.jar", "jars/singlestore-jdbc-client-1.2.1.jar"),
    ("https://repo1.maven.org/maven2/com/singlestore/singlestore-spark-connector_2.12/4.1.5-spark-3.5.0/singlestore-spark-connector_2.12-4.1.5-spark-3.5.0.jar", "jars/singlestore-spark-connector_2.12-4.1.5-spark-3.5.0.jar"),
    ("https://repo1.maven.org/maven2/org/apache/commons/commons-dbcp2/2.12.0/commons-dbcp2-2.12.0.jar", "jars/commons-dbcp2-2.12.0.jar"),
    ("https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.12.0/commons-pool2-2.12.0.jar", "jars/commons-pool2-2.12.0.jar"),
    ("https://repo1.maven.org/maven2/io/spray/spray-json_3/1.3.6/spray-json_3-1.3.6.jar", "jars/spray-json_3-1.3.6.jar")
]

for url, destination in jar_urls:
    download_jar(url, destination)

print("JAR files downloaded successfully")

JAR files downloaded successfully


In [10]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = (SparkSession
             .builder
             .config("spark.jars", ",".join([destination for _, destination in jar_urls]))
             .appName("Spark Webinar")
             .getOrCreate()
        )

spark.sparkContext.setLogLevel("ERROR")

24/04/16 15:54:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 15:54:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
import nltk
import random
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

# Download NLTK
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("omw")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/jovyan/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

In [12]:
# Define the directory to save the files
output_dir = "data"

# Generate meaningful sentences
def generate_meaningful_sentence():
    # Choose a random set of synonyms from WordNet
    synset = random.choice(list(wn.all_synsets()))

    # Generate a sentence
    definition = synset.definition()
    tokens = word_tokenize(definition)

    # Capitalise the first word and end with a period
    tokens[0] = tokens[0].capitalize()
    tokens[-1] = tokens[-1] + "."

    return " ".join(tokens)

# Number of files to generate
num_files = 5

# Number of sentences in each file
num_sentences_per_file = 1

# Generate text files
for i in range(num_files):
    file_path = os.path.join(output_dir, f"file_{i+1}.txt")
    with open(file_path, "w") as file:
        for _ in range(num_sentences_per_file):
            # Generate a meaningful sentence
            sentence = generate_meaningful_sentence()
            file.write(sentence + "\n")

In [13]:
%%bash

for file in data/*.txt; do
    echo "File: $file"
    cat "$file"
    echo "----------------------"
done

File: data/file_1.txt
Jump on skis.
----------------------
File: data/file_2.txt
Hybrid willow usually not strongly weeping in habit.
----------------------
File: data/file_3.txt
Wide-ranging light-brown frog of moist North American woodlands especially spruce.
----------------------
File: data/file_4.txt
A tear gas that is stronger than CN gas but wears off faster ; can be deployed by grenades or cluster bombs ; can cause skin burns and fatal pulmonary edema.
----------------------
File: data/file_5.txt
Indicating continuing action ; continuously or steadily.
----------------------


In [14]:
import getpass

host = "svc-201fd973-4b68-402d-aa09-0bf89add18eb-dml.aws-oregon-4.svc.singlestore.com"
port = "3306"
cluster = host + ":" + port

password = getpass.getpass("Password:")

Password: ········


In [15]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [19]:
%%sql
USE spark_demo;

DROP TABLE IF EXISTS comments;
CREATE TABLE IF NOT EXISTS comments (
     value TEXT,
     file_name TEXT,
     embedding VECTOR(1536) NOT NULL
);

++
||
++
++

In [20]:
from pyspark.sql.functions import input_file_name, udf
from pyspark.sql.types import StringType

openai.api_key = os.environ.get("OPENAI_API_KEY")

# Generate embeddings for text
def generate_embeddings(text):
    # Generate embeddings for text using OpenAI
    return openai.Embedding.create(
        input = text,
        engine = "text-embedding-3-small"
    ).data[0].embedding

# Register the function as a UDF
generate_embeddings_udf = udf(generate_embeddings, StringType())

In [21]:
import time

input_dir = output_dir

# Read from the directory
df = (spark.readStream
    .format("text")
    .option("path", input_dir)
    .load()
    .withColumn("file_name", input_file_name())
)

# Apply the function to the DataFrame to generate embeddings for each row
df_with_embeddings = df.withColumn("embedding", generate_embeddings_udf("value"))

# Write each batch of data to SingleStore
def write_to_singlestore(df_with_embeddings, epoch_id):
    (df_with_embeddings.write
         .format("singlestore")
         .option("loadDataCompression", "LZ4")
         .mode("append")
         .save("spark_demo.comments")
    )

# Write the streaming DataFrame to SingleStore using foreachBatch
query = (df_with_embeddings.writeStream
    .foreachBatch(write_to_singlestore)
    .start()
)

# Wait for the query to finish processing
while query.isActive:
    time.sleep(1)
    if not query.status["isDataAvailable"]:
        query.stop()

In [22]:
%%sql
USE spark_demo;

SELECT
    SUBSTR(value, 1, 30) AS value,
    SUBSTR(file_name, LENGTH(file_name) - 9) AS file_name,
    SUBSTR(embedding, 1, 50) AS embedding
FROM comments;

value,file_name,embedding
Indicating continuing action ;,file_5.txt,"[0.0237321071,0.0341514125,0.0182303879,0.03942219"
Wide-ranging light-brown frog,file_3.txt,"[-0.00194031477,-0.0134520773,0.0547336452,0.01292"
Jump on skis.,file_1.txt,"[0.030417813,0.0341594741,0.0108481226,-0.03507469"
A tear gas that is stronger th,file_4.txt,"[0.0302113201,0.0374153629,0.00788259227,0.0550957"
Hybrid willow usually not stro,file_2.txt,"[0.0352415368,0.0366364606,-0.00554679148,0.038584"


In [23]:
# Define the text for which you want an embedding
text = "frog"

# Request the embedding
response = openai.Embedding.create(
    input = text,
    engine= "text-embedding-3-small"
)

# Get the embedding from the response
query_string = str(response.data[0].embedding)

In [24]:
%config SqlMagic.named_parameters = True

In [25]:
%%sql
USE spark_demo;

SELECT
    SUBSTR(value, 1, 30) AS value,
    SUBSTR(file_name, LENGTH(file_name) - 9) AS file_name,
    embedding <-> :query_string AS similarity
FROM comments
ORDER BY similarity
LIMIT 5;

value,file_name,similarity
Wide-ranging light-brown frog,file_3.txt,1.1131780208445137
Jump on skis.,file_1.txt,1.2943939467682306
Indicating continuing action ;,file_5.txt,1.3149931253318798
Hybrid willow usually not stro,file_2.txt,1.319062131851387
A tear gas that is stronger th,file_4.txt,1.3730401463567572


In [26]:
spark.stop()